In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from classification_1_parsing_parrotorno import dict_bag_words

In [2]:
def bag_word(id_map):
    pass

# Create a SQL connection to our SQLite database
con = sqlite3.connect("DATABASES/project.db")

cur = con.cursor()

ana_df = pd.read_sql_query("SELECT * from classification_1_analysis", con)
map_df= pd.read_sql_query("SELECT * from mapping_cites", con)

In [3]:
fig, ax = plt.subplot(7, 8,figsize= (8,6), sharey = True, sharex = True)

for i in range(1,len(map_df)+1,1):
    sbplt = ax[i%8, math.floor(i/8)]
    words=bag_word(i)
    list_tot={}
    for word in words :
        total=ana_df[word].sum()
        list_tot.append(total)
    
    sbplt.bar(words, List_tot)
#     df = pd.DataFrame({'lab':['A', 'B', 'C'], 'val':[10, 30, 20]})
#     sbplt = plt.plot.bar(x='lab', y='val', rot=0)
    sbplt.set_title(f"{map_df.at(i, "scientific_name_cites")}")

,id,scientific_name_cites,common_name,region,danger_status_UCIN,slang,annex_number_CITES,order,family
0,1,Amazona arausiaca,Psittacus arausiacus; Amazone de Bouquet; Amaz...,Dominique,VU - Vulnerable,None,1,Psittaciformes,Psittacidae
1,2,Amazona auropalliata,Psittacus (amazona) auro-palliatus; Amazona oc...,Costa Rica,EN - Endangered,None,1,Psittaciformes,Psittacidae
2,3,Amazona barbadensis,Psittacus barbadensis Gmelin; Amazone à épaule...,Venezuela,VU - Vulnerable,None,1,Psittaciformes,Psittacidae
3,4,Amazona brasiliensis,Amazone à joues bleues; Red-tailed Amazon; Pap...,Brésil,NT - Near Threatened,None,1,Psittaciformes,Psittacidae
4,5,Amazona finschi,Amazone à couronne lilas; Lilac-crowned Amazon...,Mexique,EN - Endangered,None,1,Psittaciformes,Psittacidae
...,...,...,...,...,...,...,...,...,...
59,60,Agapornis swindernianus,Inséparable à collier noir; Inséparable de Swi...,Afrique équatoriale,LC - Least Concern,None,2,Psittaciformes,Psittacidae
60,61,Agapornis fischeri,Inséparable de Fischer; Perruche de Fischer; F...,Tanzanie,NT - Near Threatened,None,2,Psittaciformes,Psittacidae
61,62,Agapornis personatus,Inséparable masqué; Yellow-collared Lovebird;,Tanzanie,LC - Least Concern,None,2,Psittaciformes,Psittacidae
62,63,Agapornis lilianae,Inséparable de Lilian; Lilian's Lovebird; Nyas...,Tanzanie; Malawi; Mozambique; Zambie; Zimbabwe,NT - Near Threatened,None,2,Psittaciformes,Psittacidae
